In [1]:
import pandas as pd
from td.client import TDClient
import requests
import json
from datetime import date
import time
import urllib
import requests
from splinter import Browser
from config import client_id, username, password, redirect_url
from config import question1, answer1, question2, answer2, question3, answer3, question4, answer4

In [2]:
# Define the path to your browser
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}

In [3]:
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
method = 'GET'
url = 'https://auth.tdameritrade.com/auth?'
client_code = client_id + '@AMER.OAUTHAP'

In [5]:
# Define Payload, MAKE SURE TO HAVE THE CORRECT REDIRECT URI
payload_auth = {'response_type': 'code', 'redirect_uri': 'http://127.0.0.1', 
                'client_id': client_code}

built_url = requests.Request(method, url, params=payload_auth).prepare()

In [6]:
# Go to the URL
my_url = built_url.url
browser.visit(my_url)

In [7]:
# Fill Out the Form
payload_fill = {'username': username, 'password': password}
browser.find_by_id('username0').first.fill(payload_fill['username'])
browser.find_by_id('password1').first.fill(payload_fill['password'])
browser.find_by_id('accept').first.click()
time.sleep(1)

In [8]:
# Get the Text Message Box
browser.find_by_text('Can\'t get the text message?').first.click()

# Get the Answer Box
browser.find_by_value("Answer a security question").first.click()

In [9]:
# Answer the Security Questions.
if browser.is_text_present(question1):
    browser.find_by_id('secretquestion0').first.fill(answer1)

elif browser.is_text_present(question2):
    browser.find_by_id('secretquestion0').first.fill(answer2)

elif browser.is_text_present(question3):
    browser.find_by_id('secretquestion0').first.fill(answer3)

elif browser.is_text_present(question4):
    browser.find_by_id('secretquestion0').first.fill(answer4)

In [10]:
# Submit results
browser.find_by_id('accept').first.click()

In [11]:
# Trust this device
browser.find_by_xpath('/html/body/form/main/fieldset/div/div[1]/label').first.click()
browser.find_by_id('accept').first.click()

In [12]:
# Sleep and click Accept Terms.
time.sleep(5)
browser.find_by_id('accept').first.click()
time.sleep(5)

In [13]:
new_url = browser.url
parse_url = urllib.parse.unquote(new_url.split('code=')[1])
browser.quit()

In [14]:
# THE AUTHENTICATION ENDPOINT

# define the endpoint
url = r"https://api.tdameritrade.com/v1/oauth2/token"

# define the headers
headers = {"Content-Type":"application/x-www-form-urlencoded"}

# define the payload
payload = {'grant_type': 'authorization_code', 
           'access_type': 'offline', 
           'code': parse_url, 
           'client_id': client_id, 
           'redirect_uri': 'http://127.0.0.1'}

# post the data to get the token
authReply = requests.post(r'https://api.tdameritrade.com/v1/oauth2/token', headers = headers, data = payload)

# convert it to a dictionary
decoded_content = authReply.json()                       

In [15]:
# grab the access_token
access_token = decoded_content['access_token']
headers = {'fields': 'fields=positions',
           'Authorization': "Bearer {}".format(access_token)}

In [16]:
# ACCOUNT ENDPOINT

# define an endpoint with a stock of your choice, MUST BE UPPER
endpoint = r"https://api.tdameritrade.com/v1/accounts/{}".format('497083264')

# define the payload
#payload = {'apikey': api_key}

# make a request
content = requests.get(url = endpoint, headers = headers)

# convert it dictionary object
data = content.json()

In [17]:
cashAvail = data['securitiesAccount']['initialBalances']['cashAvailableForTrading']

In [18]:
acctValue = data['securitiesAccount']['initialBalances']['accountValue']

In [19]:
assetValue = data['securitiesAccount']['initialBalances']['longStockValue']

In [20]:
tdValues = {'Date': date.today(),
            'TD_Cash': cashAvail,
            'TD_AccountValue': acctValue,
            'TD_Asset Value': assetValue}

tdValues = pd.DataFrame.from_dict(tdValues, orient = 'index').T
tdValues.set_index('Date', inplace = True)